In [1]:
import tensorflow as tf
from tensorflow.keras import layers, Model, Sequential
from tensorflow.keras.applications import ResNet50, VGG19
from tensorflow.keras.layers import Input, Dense, Flatten, Concatenate, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

# Step 1: Mount Google Drive


# Step 2: Define Image Directories


# Step 3: Data Preprocessing and Augmentation


# Training and validation generators



# Step 4: Build the Hybrid Model


# Custom CNN model


# CNN branch


# ResNet50 branch


# VGG19 branch

# Combine features


# Dense layers for classification


# Create and compile the model


# Step 5: Train the Model


# Step 6: Evaluate the Model


# Step 7: Save the Model


# Step 8: Plot Training and Validation Results



In [2]:
from google.colab import drive
drive.mount('/gdrive')

KeyboardInterrupt: 

In [ ]:
data_dir = '/gdrive/MyDrive/Oral Cancer'

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.3  # 70% training, 30% validation split
)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',  # Change this to 'categorical' if more than 2 classes
    subset='training'
)


In [ ]:
val_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

In [ ]:
input_shape = (224, 224, 3)
inputs = Input(shape=input_shape)

In [ ]:
def build_custom_cnn(input_tensor):
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(input_tensor)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Flatten()(x)
    return x

In [ ]:
cnn_features = build_custom_cnn(inputs)

In [ ]:
resnet = ResNet50(weights="imagenet", include_top=False, input_tensor=inputs)
resnet_features = Flatten()(resnet.output)

In [ ]:
vgg = VGG19(weights="imagenet", include_top=False, input_tensor=inputs)
vgg_features = Flatten()(vgg.output)

In [ ]:
combined_features = Concatenate()([cnn_features, resnet_features, vgg_features])

In [ ]:
x = Dense(512, activation='relu')(combined_features)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
outputs = Dense(1, activation='sigmoid')(x)  # Use 1 for binary classification (Cancer/Non-cancer)

In [ ]:
hybrid_model = Model(inputs=inputs, outputs=outputs)
hybrid_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = hybrid_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    verbose=1
)

In [ ]:
hybrid_model.evaluate(val_generator)

In [ ]:
hybrid_model.save("/gdrive/MyDrive/hybrid_oral_cancer_model.h5")

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend()
plt.title('Accuracy')

plt.subplot(1, 2, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend()
plt.title('Loss')

plt.show()